In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal
import matplotlib.pyplot as plt

In [2]:
def f(x,y):
    return 418.9829*2 - x*np.sin(np.sqrt(abs(x))) - y*np.sin(np.sqrt(abs(y)))

In [50]:
f.__code__.co_argcount

2

### Estructura cromosómica

In [728]:
class Gen:
    def __init__(self,dominio,precision):
        """
        Parameters
        ----------
        dominio : tuple
            Límites (liminf, limsup) de la variable en cuestión.
        precision : int
            Número de cifras decimales luego del punto para la variable en cuestión. Debe ser mayor al máximo número de decimales en el dominio
        """       
        self.dominio = dominio
        self.precision = precision
        self.len = self._length()

        # Offset lb -> all(0) ; ub -> all(1)
        self.norm = (Decimal(str(dominio[1])) - Decimal(str(dominio[0]))) *10**precision / int('1'*self.len,2)
        self.offset = Decimal(str(dominio[0])) #* 10**precision

    
    def _length(self):
        """Calcula el número de bits en un gen.

        Returns
        -------
        int
            Número de bits del gen <-> variable.
        """
        # Limites
        rango = self.dominio[1] - self.dominio[0]
        cell_count = math.ceil(np.log2(rango))

        # Decimales
        cell_count += math.ceil(np.log2(10**self.precision - 1))
        return cell_count
    
    def translate(self,x,mode='decode'):
        '''
        Traductor para codificar/decodificar (reales <-> binario) genes. 

        Parameters
        ----------
        x : float or str
            Gen a traducir (float si es un real, str si es binario).
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str or float
            Gen traducido (str si x era un real, float si x era binario).
        '''
        
        if mode == 'code':
            ## Real -> Binario
            
            # Cortar decimales por redondeo
            x = float(x) # failsafe
            x = round(x,self.precision)

            # Eliminar punto
            x_int,x_dec = str(x).split(".")
            x_dec = x_dec.ljust(self.precision,"0") # fill precision

            # Representación entera
            x_rep = int(x_int + x_dec)

            # Offset -> ran >= 0
            x_rep = (Decimal(str(x_rep)) - self.offset) / self.norm
            x_rep = round(x_rep) # Nuevos decimales son de orden mayor a la precision
            return np.binary_repr(x_rep, width=self.len)
        
        elif mode == 'decode':
            ## Binario -> Real

            # Convierte a entero base 10
            x_rep = int(x,2)

            # Extraer Normalización
            x_rep = round(x_rep * self.norm)  # Nuevos decimales son de orden mayor a la precision + self.offset
            x_rep = str(x_rep).zfill(self.precision)

            # Añadir el punto decimal
            int_part = x_rep[:-self.precision]
            dec_part = x_rep[-self.precision:]
            x_float = Decimal(int_part + '.' + dec_part)
            
            # Regresar offset
            x_float = x_float + self.offset 
            return float(x_float)

In [597]:
class Cromosoma:
    def __init__(self,*genes):
        """
        Parameters
        ----------
        genes : Gen
            Gen(es) del cromosoma ordenado(s)
        """       
        self.genes = genes
        self.crom_len = sum([gen.len for gen in genes])

    def representacion(self,x,mode='decode'):
        '''
        Traductor para codificar/decodificar (reales <-> binario) cromosomas a través de sus genes.

        Parameters
        ----------
        x : list o str
            Expresión del cromosoma (lista si los genes se expresan en números reales, str si el cromosoma es binario)
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str o list
            Cromosoma traducido (elementos tipo str si "genes" era una lista, lista si "genes" era binario).
        '''

        if mode == 'code':
            ## Real -> Binario
            genes_bin = [gen.translate(val, mode=mode) for val,gen in zip(x,self.genes)]
            self.cromosoma = ''.join(genes_bin)
            return self.cromosoma
        
        elif mode == 'decode':
            ## Binario -> Real
            genes_lens = [gen.len for gen in self.genes]
            ind_final = np.cumsum(genes_lens)
            genes_r = [gen.translate(x[ind-lens:ind]) for gen, lens, ind in zip(self.genes, genes_lens, ind_final)]
            return genes_r

### Algoritmo

In [ ]:
for generation in range(num_generations):
    # Calcular la aptitud para cada individuo en la población
    fitness_scores = [fitness_function(float(individual[:7]), float(individual[7:])) for individual in population]
    
    # Retener al mejor individuo
    max_fitness_index = np.argmax(fitness_scores)
    if best_fitness is None or fitness_scores[max_fitness_index] > best_fitness:
        best_fitness = fitness_scores[max_fitness_index]
        best_individual = population[max_fitness_index]
    
    # Estadísticas de la generación
    print(f"Generación {generation + 1}:")
    print(f"  Máximo de aptitud: {max(fitness_scores)}")
    print(f"  Media de aptitud: {np.mean(fitness_scores)}")
    print(f"  Mínimo de aptitud: {min(fitness_scores)}")
    print(f"  Número de cruzas: {population_size // 2}")
    print(f"  Número de mutaciones: {int(mutation_rate * population_size)}")
    print(f"  Mejor individuo: {best_individual}, aptitud: {best_fitness}")
    
    # Selección de los padres para la reproducción
    selected_indices = np.random.choice(range(population_size), size=population_size, replace=True, p=fitness_scores / np.sum(fitness_scores))
    selected_population = [population[i] for i in selected_indices]
    
    # Crear la nueva generación mediante cruce y mutación
    new_population = []
    for i in range(0, population_size, 2):
        parent1, parent2 = selected_population[i], selected_population[i+1]
        child1, child2 = crossover(parent1, parent2)
        if np.random.rand() < mutation_rate:
            child1 = mutation(child1)
        if np.random.rand() < mutation_rate:
            child2 = mutation(child2)
        new_population.extend([child1, child2])
    
    population = new_population

In [743]:
class SimpleEvolutionary(Cromosoma):

    def optimize(self,f,N,num_gener):
        self.f = f
        self.N = N
        self.mejor_individuo = None

        if N % 2 != 0: # Sólo poblaciones pares
            self.N = N + 1

        # Genera población aleatoria
        self.poblacion = self.random_generator(self.N,self.crom_len)

        for generacion in range(num_gener):
            # Selecciona padres
            par_selection = self.ruleta()
            # Cruza los padres 2 a 2
            mothers = par_selection.iloc[1::2]
            fathers = par_selection.iloc[::2]
            
            # Generación de hijos
            cruza = np.array(np.vectorize(self.cruza)
                            (mothers['cromosoma'],fathers['cromosoma'])
                            ).reshape(-1,1)
            mutacion = np.vectorize(self.mutacion)(cruza)
            
            # Aptitud de hijos
            descendencia = pd.DataFrame([self.individuo(cromosoma[0]) for cromosoma in mutacion],
                                        columns = ['cromosoma','apt'])
            # Elitismo
            nueva_poblacion = self.sobrevivientes(descendencia)
            self.poblacion = nueva_poblacion
            self.mejor_individuo = nueva_poblacion.iloc[-1]
            
            if self.mejor_individuo['apt'] == self.poblacion['apt'].iloc[0]:
                print(generacion)
                return self.mejor_individuo

        return self.poblacion
    

    ##############################
    #### GENERACIÓN ALEATORIA ####
    def individuo(self,cromosoma):
        return cromosoma, self.f(*self.representacion(cromosoma))
    
    def random_generator(self,N,size):
        '''
        Rutina de generación aleatoria de individuos

        Parameters
        ----------
        N : int
            Número de individuos en la población.
        size : int
            Longitud del cromosoma característico de los individuos en la población.

        Returns
        -------
        array
            Individuos generados como un array de cromosomas (col0) con su aptitud (col1) respectiva.
        '''
        # Creación de población de individuos
        poblacion = np.random.binomial(n=1, p=0.5, size=(N,size)).astype(str)
        poblacion = pd.DataFrame([self.individuo(''.join(cromosoma)) for cromosoma in poblacion],
                                 columns = ['cromosoma','apt']) 

        return poblacion
    

    ###############################
    #### SELECCIÓN: POR RULETA ####

    def ruleta(self):
        '''
        Selección de candidatos a reproducción de la población actual.

        Returns
        -------
        DataFrame
            Individuos seleccionados para reproducirse con la misma dimensión que la población.
        '''
        # Cálculo de proporción de aptitud y límites de probabilidad acumulada
        pi = self.poblacion['apt'] / self.poblacion['apt'].sum()
        pop_lims =  pd.DataFrame({'liminf': (pi).cumsum() - pi,
                                  'limsup': (pi).cumsum()})
        
        # Selección aleatoria de candidatos (lambda = |población|)
        R = np.random.uniform(0,1,pop_lims.shape[0])
        k_pop = pd.DataFrame(map(lambda x: self.poblacion[(pop_lims['liminf'] < x) & (x <= pop_lims['limsup'])].iloc[0], R))
        
        return k_pop
    
    ########################################
    #### OPERADOR CRUZA: CRUZA UN PUNTO ####
    def cruza(self,par1,par2,cr=0.8):
        fail = np.random.uniform(0,1)

        if fail < cr:
            crossover_point = np.random.randint(0, len(par1)) 
            ch1 = par1[:crossover_point] + par2[crossover_point:]
            ch2 = par2[:crossover_point] + par1[crossover_point:]
        else:
            ch1 = par1
            ch2 = par2
        return ch1, ch2
    
    ##############################################
    #### OPERADOR MUTACIÓN: INVERSIÓN DE BITS ####
    def mutacion(self,cromosoma,mr=None):
        if not mr:
            mr = 1 / len(cromosoma)
        flip = np.random.binomial(1,mr,len(cromosoma))
        mutated_cr = [str((int(d) + f) % 2) for d,f in zip(cromosoma,flip)]
        mutated_cr = ''.join(mutated_cr)
        return mutated_cr
    
    #####################################
    #### SIGUIENTE POBLACIÓN (mu,mu) ####
    def sobrevivientes(self,descendientes):
        # Mejor de la generación anterior
        elite_ind = self.poblacion.sort_values('apt').iloc[-1]
        # Intercambiar el mejor de la generación anterior por el peor de la actual
        descendientes = descendientes.sort_values('apt')
        descendientes.iloc[0] = elite_ind
        return descendientes

### Test

In [729]:
# arbitrary number of genes
Gen1 = Gen((-500,500),5)
Gen2 = Gen((-500,500),3)
CromA = Cromosoma(Gen1,Gen2)

In [744]:
se = SimpleEvolutionary(Gen1,Gen2)
se.optimize(f,1000,1000)

10


cromosoma    00010100101000100001010001100010100001100110110
apt                                              1675.620116
Name: 381, dtype: object

In [706]:
se = SimpleEvolutionary(Gen1,Gen2)
se.f = f
se.poblacion = se.random_generator(10,se.crom_len)
# se.optimize(f,100,1000000)
pars = se.ruleta()
pars

,cromosoma,apt
2,11010010111000001001111110000001110011000101000,3.042509e+07
9,10111010100110101110111111100111110010101111001,3.042516e+07
7,10101100000110011101110001000010011110100101010,3.042510e+07
3,00111101000111101001101101101111011110100100101,3.042522e+07
6,11001111101110010011011001101110101000000101110,3.042523e+07
8,11101001110101101011010111001110001110110111101,3.042523e+07
2,11010010111000001001111110000001110011000101000,3.042509e+07
7,10101100000110011101110001000010011110100101010,3.042510e+07
2,11010010111000001001111110000001110011000101000,3.042509e+07
1,01100110110010000010100011000110110010101000000,3.042514e+07


In [352]:
pars['n'] = np.arange(pars.shape[0]) % 2
pars

,cromosoma,apt,n
6,00101111001011100010001010101000111100000111001,725.612173,0
8,10100010011111111000100000100010001101110111100,1355.164338,1
0,11111101000110011010010111111010011000100001010,1134.778507,0
1,01111111110111011101000001100100110011010111010,788.301409,1
0,11111101000110011010010111111010011000100001010,1134.778507,0
0,11111101000110011010010111111010011000100001010,1134.778507,1
6,00101111001011100010001010101000111100000111001,725.612173,0
2,11011101000010010110100000000011011100111011100,1080.276636,1
6,00101111001011100010001010101000111100000111001,725.612173,0
3,00000101101111101110001111111101111101110011011,514.698847,1


In [473]:
mothers = pars.iloc[1::2]
fathers = pars.iloc[::2]

In [355]:
mothers

,cromosoma,apt
5,11011001010111000010010100110100110001101111001,946.688074
5,11011001010111000010010100110100110001101111001,946.688074
4,11100111001011011011011100010111101111000000001,419.800253
7,11001101110111011001101000010010010101010100000,1081.690424
8,10100010011111111000100000100010001101110111100,1355.164338


In [356]:
fathers

,cromosoma,apt
2,11011101000010010110100000000011011100111011100,1080.276636
5,11011001010111000010010100110100110001101111001,946.688074
8,10100010011111111000100000100010001101110111100,1355.164338
7,11001101110111011001101000010010010101010100000,1081.690424
9,00000100101110001101111111100000101001010111001,902.055000


In [499]:
desc = np.array(np.vectorize(se.cruza)(mothers['cromosoma'],fathers['cromosoma'])).reshape(-1,1)
desc

array([['10100010011111111000100000100010001101110111100'],
       ['11011001010111000010010100110100110001101111001'],
       ['01111111110111011101000001100100101001010111001'],
       ['00101111001011100010000000100010001101110111100'],
       ['11100111001011011011011100010111101111000000001'],
       ['11100111001011011011011100010111101111000000001'],
       ['01111111110111011101000001100100110011010111010'],
       ['00000100101110001101111111100000110011010111010'],
       ['10100010011111111000101010101000111100000111001'],
       ['11001101110111011001101000010010010101010100000']], dtype='<U47')

In [532]:
mut = np.vectorize(se.mutacion)(desc)
mut

array([['10100010010111111100100000000010001101110111100'],
       ['11011001010111000010010110111100110001101111001'],
       ['01111111110111011101000001100100101001010111001'],
       ['00101111001011100010000000100010001101110110100'],
       ['11100111001011011011011100010111101111000000001'],
       ['11100111001011011011011100010111101111000000101'],
       ['01111011110111011101000001100100110011010110010'],
       ['00000100101110001101111111100000110011010111010'],
       ['10100010011111111000101010101000111100000111001'],
       ['11001101110111011001101000010010011101010100000']], dtype='<U47')

In [534]:
np.vectorize(se.individuo)(mut)

(array([['10100010010111111100100000000010001101110111100'],
        ['11011001010111000010010110111100110001101111001'],
        ['01111111110111011101000001100100101001010111001'],
        ['00101111001011100010000000100010001101110110100'],
        ['11100111001011011011011100010111101111000000001'],
        ['11100111001011011011011100010111101111000000101'],
        ['01111011110111011101000001100100110011010110010'],
        ['00000100101110001101111111100000110011010111010'],
        ['10100010011111111000101010101000111100000111001'],
        ['11001101110111011001101000010010011101010100000']], dtype='<U47'),
 array([[1356.35739438],
        [ 534.20522981],
        [ 833.58681975],
        [ 966.25812639],
        [ 419.80025344],
        [ 419.83030224],
        [ 775.61873323],
        [ 954.14737835],
        [1114.50131864],
        [1091.04627051]]))

In [340]:
pd.Series(('123411', 1231), index=['aa','bb'])

aa    123411
bb      1231
dtype: object

In [105]:
N=10
size=CromA.crom_len
poblacion = np.random.binomial(n=1, p=0.5, size=(N,size)).astype(str)
poblacion = np.array([''.join(individuo) for individuo in poblacion]) 

# Cálculo de aptitud
apts = np.array([f(*CromA.representacion(individuo,mode='decode')) for individuo in poblacion])

np.array(list(zip(poblacion, apts)))[:,0]

array(['10111010010010010000100011011001111111110111110',
       '00101000001111111001101011000011110001100110110',
       '01100011000000101110011011011110101100101110001',
       '01010011010111000100000010000111010000101011110',
       '11010001011011110001110111111000111000110011011',
       '01010100000000110010100111111011100000100010110',
       '00010111000110100110111001101000100110111111010',
       '11011000100111100110100110110101101001100101111',
       '00010011000001000001111001111001011000100100111',
       '10100110000011111111110010001011001100011010010'], dtype='<U47')

In [751]:
N=10
size=CromA.crom_len
poblacion = np.random.binomial(n=1, p=0.5, size=(N,size)).astype(str)
poblacion = np.array([''.join(individuo) for individuo in poblacion]) 

# Cálculo de aptitud
apts = np.array([f(*CromA.representacion(individuo,mode='decode')) for individuo in poblacion])

poblacion, apts

(array(['10000100010100001000111111010011101000011111010',
        '00000001110001111011100111000101000000101110101',
        '11111100110101110010000111000110111101001101000',
        '00000101111111000111110111111111101011100010110',
        '10101111011101011000000110111111011011000010100',
        '00000110111011011001010110010010100000110100100',
        '01000100111100100000010001000011100111001010101',
        '01000111111101011000000011111001000110101100000',
        '01000101000110011000100001101011011100000010000',
        '01101000010111010000001011010010001111110100010'], dtype='<U47'),
 array([ 958.79303455,  625.91661537,  630.40468358,  984.5514737 ,
         659.05638712,  910.30315816, 1237.15023152, 1271.91560023,
         872.60493185,  760.32974113]))

In [752]:
individuos = SimpleEvolutionary(Gen1,Gen2).optimize(f,10)
individuos

TypeError: SimpleEvolutionary.optimize() missing 1 required positional argument: 'num_gener'

In [162]:
R = np.random.uniform(0,1,individuos['apt'].shape[0])

In [160]:
individuos[(300 < individuos['%']) & (individuos['%'] <= 700)]

,cromosoma,apt
3,01111100111011100101000111000101110010100001000,568.013314
4,10111100000000111111100000110111100011010010110,667.317125
8,11101011011010010011110111010010001001101110111,355.979157


In [163]:
lims = pd.DataFrame({'%':(individuos['apt'] / individuos['apt'].sum()),
              'lf': (individuos['apt'] / individuos['apt'].sum()).cumsum() - (individuos['apt'] / individuos['apt'].sum()),
              'ls': (individuos['apt'] / individuos['apt'].sum()).cumsum()})
lims

,%,lf,ls
0,0.124991,0.000000,0.124991
1,0.086723,0.124991,0.211714
2,0.100956,0.211714,0.312670
3,0.064218,0.312670,0.376888
4,0.075445,0.376888,0.452333
5,0.161702,0.452333,0.614036
6,0.112264,0.614036,0.726299
7,0.145911,0.726299,0.872210
8,0.040246,0.872210,0.912456
9,0.087544,0.912456,1.000000


In [183]:
pd.DataFrame(map(lambda x: individuos[(lims['lf'] < x) & (x <= lims['ls'])].iloc[0], R))

,cromosoma,apt,%
4,10111100000000111111100000110111100011010010110,667.317125,0.075445
2,11011000110100000111111000101111010011001001011,892.958987,0.100956
2,11011000110100000111111000101111010011001001011,892.958987,0.100956
8,11101011011010010011110111010010001001101110111,355.979157,0.040246
7,10100010100110101101111010000011010101000000111,1290.590836,0.145911
0,10001011010001100010000100111001010011001000011,1105.547877,0.124991
2,11011000110100000111111000101111010011001001011,892.958987,0.100956
7,10100010100110101101111010000011010101000000111,1290.590836,0.145911
2,11011000110100000111111000101111010011001001011,892.958987,0.100956
1,10100110110011110110110110110101100110110111001,767.071722,0.086723


In [171]:
individuos[(lims['lf'] < 0.23970917) & (0.23970917 <= lims['ls'])]

,cromosoma,apt,%
2,11011000110100000111111000101111010011001001011,892.958987,0.100956


In [753]:
N = 30
crom_len = CromA.crom_len
pp = np.random.binomial(n=1, p=0.5, size=(N,crom_len))

pp = pp.astype(str)
pp_bin = np.array([''.join(xi) for xi in pp])

In [754]:
_realrep = np.array([CromA.representacion(cr,mode='decode') for cr in pp_bin])
__binrep = np.array([CromA.representacion(cr,mode='code') for cr in _realrep])

for i,j in zip(pp_bin,__binrep):
    if i!=j:
        print('i: ', CromA.representacion(i,mode='decode'), i)
        print('j: ', CromA.representacion(j,mode='decode'), j)
        print('#####################')

i:  [-119.61948, 265.793] 01100001011000001001111000111000100000010101111
j:  [380.38552, -233.707] 11100001011000001111001000001000100001010111100
#####################
i:  [-285.75998, 364.713] 00110110110110000110111100111011101010111011100
j:  [214.24503, -134.787] 10110110110110001100001100001011101011111101001
#####################
i:  [-210.54781, 270.335] 01001010000110011000100111111000101001101001010
j:  [289.4572, -229.165] 11001010000110011101110111101000101010101010111
#####################
i:  [442.28939, -206.819] 11110001001110011110000010001001011000011011110
j:  [-57.70561, 293.682] 01110001001110100011010010011001011001011101011
#####################
i:  [-294.34202, 44.812] 00110100101001100000000001110001011011110001100
j:  [205.66299, -454.688] 10110100101001100101010001000001011100110011001
#####################
i:  [398.79351, -491.776] 11100110000101110101010011000000010000110101111
j:  [-101.20149, 8.725] 01100110000101111010100011010000010001110111100
#######

In [755]:
_realrep
   

array([[-119.61948,  265.793  ],
       [-285.75998,  364.713  ],
       [-210.54781,  270.335  ],
       [ 442.28939, -206.819  ],
       [-294.34202,   44.812  ],
       [ 398.79351, -491.776  ],
       [  31.08428, -438.737  ],
       [ 455.58109,  340.573  ],
       [ 483.15024, -436.13   ],
       [-332.47894,  340.227  ],
       [-110.93287, -405.422  ],
       [-314.32529,  -63.168  ],
       [  87.29124, -309.644  ],
       [ 289.88194,  180.216  ],
       [-471.46873, -329.153  ],
       [  80.92293,  -32.307  ],
       [ 438.26174,  478.579  ],
       [ 432.6314 , -341.813  ],
       [ -10.54553,   86.731  ],
       [ 171.64204, -178.876  ],
       [-228.30143,  474.033  ],
       [ 141.67061, -468.806  ],
       [-345.86044, -316.615  ],
       [ 454.47862,  359.025  ],
       [-404.24997,  443.222  ],
       [ 250.44466,  202.735  ],
       [ 400.41406,  485.604  ],
       [-247.10562,  417.547  ],
       [ 340.01288, -267.555  ],
       [-166.88131, -409.507  ]])